In [1]:
from scipy.stats import zipf, geom
import pandas as pd
import numpy as np
import random
import sys
from scipy.special import zeta

In [2]:
df = pd.read_csv('out.csv')
lis = list(df['sum'])

In [3]:
def riemann_zeta_func_deriv(x):
    h = 1e-5
    return (zeta(x+h) - zeta(x-h))/(2*h)

In [4]:
def func(k,q,a,p):
    return q*((k**-a)/(zeta(a))) + (1-q)*(((1-p)**(k-1)) * p)

In [5]:
def zeta_param_deriv(k,q,a,p):
    return (1/func(k,q,a,p)) * q * (((-np.log(k) * k**(-a) * zeta(a)) - (k**(-a) * riemann_zeta_func_deriv(a))) / (zeta(a)**2))

In [6]:
def geom_param_deriv(k,q,a,p):
    return (1/func(k,q,a,p)) * (1-q) * ((-p*(k-1)*(1-p)**(k-2))+((1-p)**(k-1)))

In [7]:
def mix_param_deriv(k,q,a,p):
    return (1/func(k,q,a,p)) * (((k**(-a))/(zeta(a)))-(((1-p)**(k-1))*p))

In [8]:
def mle(runs):
    avg_A, avg_P, avg_Q = [], [], []
    for _ in range(runs):
        a = random.random()+1 # random initial value between 1 and 2
        p = random.random() # random initial value between 0 and 1
        q = random.random() # random initial value between 0 and 1
        ep = 0.00001
        alpha = 0.001
        for _ in range(100):
            k = random.choice(lis)
            da, dp, dq = zeta_param_deriv(k,q,a,p), geom_param_deriv(k,q,a,p), mix_param_deriv(k,q,a,p)
            atmp, ptmp, qtmp = a, p, q
            a = a - alpha * da
            p = p - alpha * dp
            q = q - alpha * dq
            if (abs((a - atmp) / a) < ep) and (abs((p - ptmp) / p) < ep) and (abs((q - qtmp) / q) < ep):
                break
        if str(a) != 'nan' and str(p) != 'nan' and str(q) != 'nan':
            avg_A.append(a)
            avg_P.append(p)
            avg_Q.append(q)
    print('zeta MLE', sum(avg_A)/len(avg_A))
    print('geom MLE', sum(avg_P)/len(avg_P))
    print('mix MLE', sum(avg_Q)/len(avg_Q))

In [9]:
mle(1000)

/var/folders/2r/7vbf7sp10g52q3ngqyxjw99h0000gn/T/ipykernel_18326/2367670127.py:2: RuntimeWarning: overflow encountered in scalar power
  return q*((k**-a)/(zeta(a))) + (1-q)*(((1-p)**(k-1)) * p)
/var/folders/2r/7vbf7sp10g52q3ngqyxjw99h0000gn/T/ipykernel_18326/2163530036.py:2: RuntimeWarning: invalid value encountered in scalar multiply
  return (1/func(k,q,a,p)) * (1-q) * ((-p*(k-1)*(1-p)**(k-2))+((1-p)**(k-1)))


zeta MLE 1.5341960734201714
geom MLE 0.6802938406864648
mix MLE 0.8022702457178169
